In [ ]:
import cv2 as cv2 # this imports OpenCV
import numpy as np # this imports numpy
import matplotlib.pyplot as plt


# Sources: 
# I know wikipedia isn't a good source, but I just needed to know
# whether to use color or grayscale https://en.wikipedia.org/wiki/Harris_corner_detector

# function for Harris Corner Detector



# 1. import image and convert to gray scale
# 2. compute x derivative at point p using sobel (extrapolate w/ reflect)
# 3. compute y derivative at point p using sobel (extrapolate w/ reflect)
# 4. 5x5 gaussian mask w/ sigma = 0.5 (use reflect for values out of range)


#input the image
def HCD(image):
    #---------------- Problem 1 Task 1 ----------------------
    
    # import and grayscale the image
    grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    grayImage = np.float32(grayImage)
    
    # compute derivates using Sobel and extrapolate with reflect
    # source (https://www.geeksforgeeks.org/python-opencv-cv2-copymakeborder-method/)
    der_x = cv2.Sobel(grayImage, cv2.CV_64F, 1, 0, ksize=3, borderType=cv2.BORDER_REFLECT)
    der_y = cv2.Sobel(grayImage, cv2.CV_64F, 0, 1, ksize=3, borderType=cv2.BORDER_REFLECT)
    
    # This is for each of the products in the matrix
    xx = der_x **2
    yy = der_y **2
    xy = der_x * der_y
    
    # 5x5 Gaussian mask w/ sigma = 0.5 and reflect 
    # source (https://theailearner.com/tag/cv2-getgaussiankernel/)
    # Creates a 1-D Gaussian kernel
    gauss = cv2.getGaussianKernel(5, 0.5)
    # Creates a 2-D Gaussian kernel by multiplying by transpose
    mask = gauss * gauss.T
    
    # multiply w(p) to the derivative products of H
    # source (https://www.geeksforgeeks.org/python-opencv-cv2-copymakeborder-method/)
    wxx = cv2.filter2D(xx, -1, mask, borderType=cv2.BORDER_REFLECT)
    wyy = cv2.filter2D(yy, -1, mask, borderType=cv2.BORDER_REFLECT)
    wxy = cv2.filter2D(xy, -1, mask, borderType=cv2.BORDER_REFLECT)
    
    #---------------- Problem 1 Task 2 ----------------------
    
    # calculate determinant and trace of H to compute corner strength function
    det_H = wxx * wyy - wxy**2
    tr_H = wxx + wyy
    
    
    # Calculate c(H) using Harris & Stephens 1988 formula
    cornerStrength = det_H - 0.1*(tr_H**2)
    
    
    #---------------- Problem 1 Task 3 ----------------------
    
    # compute orientation as the angle of the gradient
    orient = np.arctan2(der_y, der_x)

    
    #---------------- Problem 1 Task 4 ----------------------
    
    # set threshold
    threshold = 0.01 * cornerStrength.max()
    
    # define neighborhood required
    neighborhood = np.ones((7, 7), dtype=np.uint8)
    
    # dilate w/ 7x7 neighborhood and select keypoints according to threshold
    # keypoint_loc are those where cornerStrength is equal to that dilation AND above the threshold
    maxx = cv2.dilate(cornerStrength, neighborhood)
    keypoint_loc = (cornerStrength == maxx) & (cornerStrength > threshold)
    
    keypoints = np.argwhere(keypoint_loc)
    
    keys = []
    
    # source (https://docs.opencv.org/3.4/d2/d29/classcv_1_1KeyPoint.html)
    for position in keypoints:
        # change the x and y to float to avoid error
        x = float(position[1])
        y = float(position[0])
    
        # Make keypoint object with x and y and add to keys list
        keypoint = cv2.KeyPoint(x, y, 1)

        keys.append(keypoint)
        
    return keys
    
#---------------- Problem 1 Task 5 ----------------------
    
# Load each image
car = cv2.imread('/Users/fneba/harris_car.jpeg')
elephant = cv2.imread('/Users/fneba/harris_elephant.jpeg')
sunflower = cv2.imread('/Users/fneba/harris_sunflower.jpg')
    
# Obtain keypoints
car_keys = HCD(car)
ele_keys = HCD(elephant)
sun_keys = HCD(sunflower)

# source (https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html)
def keyViz(image, keypoints):
    for keypoint in keypoints:
        x,y = keypoint.pt
        x = int(x)
        y = int(y)
        cv2.circle(image, (x,y), radius = 5, color = (0, 255, 0), thickness = 2)
    
    cv2.imshow("Keypoints", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
keyViz(car, car_keys)
keyViz(elephant, ele_keys)
keyViz(sunflower, sun_keys)
    

    


In [ ]:
import cv2 as cv2 # this imports OpenCV
import numpy as np # this imports numpy
import matplotlib.pyplot as plt

In [ ]:
#---------------- Problem 1 Task 6 ----------------------

# Load each image
car = cv2.imread('/Users/fneba/harris_car.jpeg')
elephant = cv2.imread('/Users/fneba/harris_elephant.jpeg')
sunflower = cv2.imread('/Users/fneba/harris_sunflower.jpg')


def HCD2(image):
    #---------------- Problem 1 Task 1 ----------------------
    
    # import and grayscale the image
    grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    grayImage = np.float32(grayImage)
    
    # compute derivates using Sobel and extrapolate with reflect
    der_x = cv2.Sobel(grayImage, cv2.CV_64F, 1, 0, ksize=3, borderType=cv2.BORDER_REFLECT)
    der_y = cv2.Sobel(grayImage, cv2.CV_64F, 0, 1, ksize=3, borderType=cv2.BORDER_REFLECT)
    
    # This is for each of the products in the matrix
    xx = der_x **2
    yy = der_y **2
    xy = der_x * der_y
    
    # 5x5 Gaussian mask w/ sigma = 0.5 and reflect 
    # source (https://theailearner.com/tag/cv2-getgaussiankernel/)
    # Creates a 1-D Gaussian kernel
    gauss = cv2.getGaussianKernel(5, 0.5)
    # Creates a 2-D Gaussian kernel by multiplying by transpose
    mask = gauss * gauss.T
    
    # multiply w(p) to the derivative products of H
    wxx = cv2.filter2D(xx, -1, mask, borderType=cv2.BORDER_REFLECT)
    wyy = cv2.filter2D(yy, -1, mask, borderType=cv2.BORDER_REFLECT)
    wxy = cv2.filter2D(xy, -1, mask, borderType=cv2.BORDER_REFLECT)
    
    #---------------- Problem 1 Task 2 ----------------------
    
    # calculate determinant and trace of H to compute corner strength function
    det_H = wxx * wyy - wxy**2
    tr_H = wxx + wyy
    
    
    # Calculate c(H) using Nobel 1998 formula from slides
    cornerStrength = det_H / (tr_H + 0.000001)
    
    
    #---------------- Problem 1 Task 3 ----------------------
    
    # compute orientation as the angle of the gradient
    orient = np.arctan2(der_y, der_x)
    
    # np computes it in radians so I will convert to degrees, just in case
    orient_deg = np.degrees(orient)
    
    #---------------- Problem 1 Task 4 ----------------------
    
    # set threshold
    threshold = 0.01 * cornerStrength.max()
    
    # define neighborhood required
    neighborhood = np.ones((7, 7), dtype=np.uint8)
    
    # dilate w/ 7x7 neighborhood and select keypoints according to threshold
    maxx = cv2.dilate(cornerStrength, neighborhood)
    keypoint_loc = (cornerStrength == maxx) & (cornerStrength > threshold)
    
    keypoints = np.argwhere(keypoint_loc)
    
    keys = []

    for position in keypoints:
        # change the x and y to float to avoid error
        x = float(position[1])
        y = float(position[0])
    
        # Make keypoint object with x and y and add to keys list
        keypoint = cv2.KeyPoint(x, y, 1)

        keys.append(keypoint)
        
    return keys




# Obtain keypoints
car_keys2 = HCD2(car)
ele_keys2 = HCD2(elephant)
sun_keys2 = HCD2(sunflower)

def keyViz(image, keypoints):
    for keypoint in keypoints:
        x,y = keypoint.pt
        x = int(x)
        y = int(y)
        cv2.circle(image, (x,y), radius = 5, color = (0, 255, 0), thickness = 2)
    
    cv2.imshow("Keypoints", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
keyViz(car, car_keys2)
keyViz(elephant, ele_keys2)
keyViz(sunflower, sun_keys2)

In [ ]:
#---------------- Problem 1 Task 6 Part 2----------------------


# Load each image
car2 = cv2.imread('/Users/fneba/Car2.jpg')
elephant2 = cv2.imread('/Users/fneba/elephant2.jpeg')
sunflower2 = cv2.imread('/Users/fneba/sunflower2.jpeg')



# Obtain keypoints
car_keys3 = HCD2(car2)
ele_keys3 = HCD2(elephant2)
sun_keys3 = HCD2(sunflower2)

def keyViz(image, keypoints):
    for keypoint in keypoints:
        x,y = keypoint.pt
        x = int(x)
        y = int(y)
        cv2.circle(image, (x,y), radius = 5, color = (0, 255, 0), thickness = 2)
    
    cv2.imshow("Keypoints", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
keyViz(car2, car_keys3)
keyViz(elephant2, ele_keys3)
keyViz(sunflower2, sun_keys3)